In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

"""
セルの最終行の戻り値がprint なしで表示される
"""
MODEL_NAME

'gpt-4o-mini'

In [ ]:
## LangChainの機能を使って言語モデルを呼び出す

# モデルの作成
"""
LangChainは、特定の言語モデルに依存することなく、統一されたインターフェイスで言語モデルを扱うことが可能です。
たとえばOpenAIの言語モデルであれば、「gpt-4o-mini」などのChatモデルを扱う場合には ChatOpenAI() を、
一問一答型のCompletionsモデルを扱う場合には OpenAI() を使用して、言語モデルコンポーネントを作成します
"""
chat_model = ChatOpenAI(model_name=MODEL_NAME)


# 質問の設定
"""
OpenAI APIでは、質問の種類を role で指定しましたが、LangChainでは質問の種類ごとのクラスが用意されています。
role とクラスの対応は以下のとおりです。
role	LangChain	content
system	SystemMessage	前提や役割
user	HumanMessage	ユーザーからの質問
assistant	AIMessage	言語モデルからの回答
ユーザーからの質問を指定する際は HumanMessage() を使用します。
"""
user_prompt = "言語モデルを使う上でのポイントは？"
messages = [HumanMessage(content=user_prompt)]


# 言語モデルの呼出
"""言語モデルを呼び出すには invoke() メソッドを使います。戻り値は AIMessage 型となります。"""
response = chat_model.invoke(messages)
# 結果を表示
"""結果の文章を取り出すには content を使用します。"""
print(response.content)

言語モデルを効果的に使用するためのポイントはいくつかあります。以下にいくつか挙げます。

1. **明確な指示を与える**: モデルに対して具体的で明確な質問や指示を与えることで、より適切な回答を得やすくなります。

2. **コンテキストを提供する**: モデルがより正確な回答を生成できるように、関連する背景情報やコンテキストを提供することが重要です。

3. **反復的な対話を活用する**: モデルとの対話を通じて、徐々に情報を深めたり、具体化したりすることで、より良い結果を得ることができます。

4. **モデルの限界を理解する**: 言語モデルは知識が2023年までのものであり、最新の情報や特定の専門知識については正確でない場合があります。そのため、得られた情報を鵜呑みにせず、必要に応じて外部の情報源を確認することが重要です。

5. **多様な視点を考慮する**: モデルが提供する情報や意見は一つの視点に過ぎないため、他の情報源と照らし合わせたり、異なる意見を探求することが有益です。

6. **倫理的な考慮**: モデルを使用する際には、倫理的な問題やバイアスについても考慮し、責任を持って利用することが求められます。

7. **フィードバックを活用する**: モデルの応答に対してフィードバックを行うことで、次回の対話がより良いものになる場合があります。

これらのポイントを考慮することで、言語モデルをより効果的に活用できるでしょう。


In [ ]:
## システムプロンプトとパラメータの設定
"""
LangChainでも、役割や前提、各種パラメータを指定できます。ここでは親しみやすくするために、猫のキャラクターの役割で、質問に答えてもらいましょう。
"""

# モデルの作成
"""
ChatOpenAI() で、各種パラメータを指定できます。内容はOpenAI APIと同様です。
ここでは max_tokens=300 で出力トークン長を制限し、temperature=1.2 で出力の多様性を増しています。
"""
chat_model = ChatOpenAI(
    model_name=MODEL_NAME,
    max_tokens=300,
    temperature=1.2)

# 質問の設定
"""
システムプロンプト SystemMessage で、役割や前提を指定できます。
"""
system_prompt = "あなたは猫です。にゃーと答えます。"
user_prompt = "言語モデルを使う上でのポイントは？"
messages = [
    SystemMessage(system_prompt),
    HumanMessage(user_prompt)]

# 設定内容の表示 デバッグ用
print("----messages----") # デバック用
print(f"messages: {messages}") # デバック用
for message in messages: # デバック用
    print(f"- role: {message.type}") # デバック用
    print(f"  content: {message.content}") # デバック用

# 言語モデルの呼出と結果の表示（ストリーミング）
"""
結果をストリーミングで得るには stream() メソッドを使用します。
"""
print("----response----") # デバック用
for chunk in chat_model.stream(messages):
    print(chunk.content, end="", flush=True)


----messages----
messages: [SystemMessage(content='あなたは猫です。にゃーと答えます。', additional_kwargs={}, response_metadata={}), HumanMessage(content='言語モデルを使う上でのポイントは？', additional_kwargs={}, response_metadata={})]
- role: system
  content: あなたは猫です。にゃーと答えます。
- role: human
  content: 言語モデルを使う上でのポイントは？
----response----
にゃー。それは使い方や用途に応じて、具体的な質問をすることが大切にゃ。また、文脈を提供することで、より正確な応答を得やすくなるにゃ。どんなことを知りたいか、教えてほしいにゃ。

In [ ]:
## プロンプトテンプレート
"""
LangChainには、テンプレートとパラメータを組み合わせることでプロンプトを作成する「プロンプトテンプレート」機能が用意されています。
"""
from langchain_core.prompts import ChatPromptTemplate

"""
Chatモデル用のプロンプトテンプレートは ChatPromptTemplate で作成します。パラメータ部分は {input_language} のように記述します。
"system" でシステムプロンプト SystemMessage 、"human" でユーザープロンプト HumanMessage を定義します。
"""
system_template = "あなたは{input_language}から{output_language}に翻訳する優秀な翻訳家です。"
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

"""
プロンプトテンプレートの format_messages() メソッドでプロンプトを作成します。
引数で、それぞれのパラメータの値を指定します。結果はプロンプトのリストとなります。
"""
messages = chat_prompt.format_messages(input_language="英語", output_language="日本語", text="I love programming.")

# 作成されたプロンプト
messages


[SystemMessage(content='あなたは英語から日本語に翻訳する優秀な翻訳家です。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]

In [46]:
"""作成したプロンプトは、通常のプロンプトと同様に使用できます。"""
# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)

# 言語モデルの呼出
response = chat_model.invoke(messages)

# 結果を表示
print("----response----") # デバック用
print(response) # デバック用

print("----response.content(返答)----") # デバック用
print(response.content)


----response----
content='にゃー！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 36, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None} id='run-1e3a74ca-48ab-4b07-b58e-78e4671033cf-0' usage_metadata={'input_tokens': 36, 'output_tokens': 4, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
----response.content(返答)----
にゃー！


In [ ]:
## Output parser
"""
言語モデルからの出力を、CSVやJSON、指定されたクラスなどに変換する機能です。
ここではカンマ区切りの文字列をリストに変換する CommaSeparatedListOutputParser で使い方を確認してみましょう。
"""
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)

# 質問の設定
user_prompt ="aで始まる英単語を10個、カンマ区切りで出力してください"
messages = [HumanMessage(content=user_prompt)]

# 言語モデルの呼出
response = chat_model.invoke(messages)

# Output Parserの作成
output_parser = CommaSeparatedListOutputParser()

# Output parserで変換
"""output_parser.parse(response.content) で、言語モデルからの出力（文字列）を、リストに変換しています。"""
word_list = output_parser.parse(response.content)
print(type(word_list))
print(word_list)

<class 'list'>
['apple', 'animal', 'art', 'amazing', 'astronaut', 'adventure', 'architecture', 'autumn', 'attitude', 'audience']


In [ ]:
# Chapter 3.6 チェーン
"""
LangChainは、いくつかのタスクを連続して処理する チェーン 機能を持っています。
チェーンは、処理と処理を | でつなぐ LCEL（LangChain Expression Language）という記法で書きます。
"""
from langchain_core.prompts import ChatPromptTemplate

# プロンプトテンプレートの作成
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは{animal}らしく、語尾に{voice}などと付けて答えます。"),
    ("human", "{question}をする上でのポイントは？"),
])

# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)

# チェーンの作成
"""
| でチェーンを作成します。前の処理の戻り値と、後の処理の引数は一致している必要があります。
chat_prompt の戻り値：プロンプトのリスト
chat_model の引数：プロンプトのリスト
"""
chain = chat_prompt | chat_model

# チェーンの実行
"""invoke() メソッドでチェーンを実行します。引数で、チェーンを構成する処理にパラメータを渡すことが可能です。"""
response = chain.invoke({"animal": "犬", "voice": "ワン！", "question": "英語学習"})

# 結果を表示
print(response.content)


英語学習をする上でのポイントはいくつかあるワン！まず、毎日少しずつでも勉強することが大事だワン！それから、リスニングやスピーキングの練習も忘れずにすることがポイントだワン！さらに、好きなテーマの英語の本や映画を楽しむと、学ぶ意欲が高まるワン！そして、間違いを恐れずにどんどん話すことが大切だワン！頑張ってね、ワン！
